In [ ]:
%matplotlib widget

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from rsradia.magnets import dipole as dipole_module
# import uti_plot
import ipywidgets
from jupyter_rs_radia import radia_viewer
import radia as rad

# Make Dipole
Create a simplified H-type dipole and show several plots of the fields and field quality

In [3]:
pole_dimensions = {
    'pole_width': 3.5,
    'pole_separation': 1.5,
    'pole_height': 1.5,
    'top_height': 1.5,
    'gap_height': 0.75,
    'leg_width': 1.5
}

dipole_x_center = 0.0
dipole_x_length = 10.0

In [4]:
dipole = dipole_module.make_dipole(pole_dimensions, dipole_x_center, dipole_x_length, trimesh_mode=0, longitudinal_divisions=10)

In [5]:
rv = radia_viewer.RadiaViewer()
rv.add_geometry('Pole', dipole)

In [6]:
rv.display()

RadiaViewer(children=(Viewer(children=(VTK(layout=Layout(margin='auto', min_width='25%', width='50%'), model_d…

# Magnet Analysis

### Hit solve in Viewer before running

In [9]:
# On Axis field profile
xp = np.linspace(-15, 15, 100)
mesh = np.stack([xp, np.zeros_like(xp).ravel(), np.zeros_like(xp).ravel()]).T

In [10]:
mesh = [list(m) for m in mesh]

In [11]:
Bz_on_mesh = rad.Fld(dipole, 'Bz', mesh)
Bz0 = rad.Fld(dipole, 'Bz', [0, 0, 0])

In [12]:
# Field at center of magnet
Bz0

-1.9376517975149714

In [13]:
plt.figure()
plt.plot(xp, np.array(Bz_on_mesh) / Bz0)
plt.xlabel('x')
plt.ylabel('$B_z / B_z^0$')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Y/Z slice at X = 0 

Plot $Bz$ on a grid in YZ in the center of the dipole a $x=0$. This meshed data can then be used to calculate the 'good field region' (GFR) that is the region in YZ where $B_z(y, z) - B_{target} < tolerance$ for some tolerance. Here we just assume the the target B field is Bz at x,y,z = 0. 

In [14]:
Y, Z = np.meshgrid(np.linspace(-pole_dimensions['pole_width'], pole_dimensions['pole_width'], 100),
                   np.linspace(-pole_dimensions['gap_height'], pole_dimensions['gap_height'], 100))

In [15]:
mesh = np.stack([np.zeros_like(Y).ravel(), Y.ravel(), Z.ravel()]).T

In [16]:
mesh = [list(m) for m in mesh]

In [17]:
Bz_on_mesh = rad.Fld(dipole, 'Bz', mesh)
Bz0 = rad.Fld(dipole, 'Bz', [0, 0, 0])

### Plot Bz on Mesh

In [20]:
fig, ax = plt.subplots(1,1, figsize=(12, 4))
ax.set_aspect('equal')
ct = ax.contourf(Y, Z, np.array(Bz_on_mesh).reshape(*Y.shape) / Bz0, levels=64)
ax.contour(Y, Z, np.array(Bz_on_mesh).reshape(*Y.shape) / Bz0, levels=[0.5, 1.0, 1.25], colors='r')
plt.colorbar(ct)
ax.set_xlabel('x (mm)')
ax.set_ylabel('y (mm)')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Setup and plot GFR on Mesh

In [18]:
def gfr(bz_mesh, bz0, gf_level=10**3):
    return np.abs(np.array(bz_mesh) - bz0) / np.abs(bz0)
    

In [19]:
gf_level = gfr(Bz_on_mesh, Bz0)

In [21]:
fig, ax = plt.subplots(1,1, figsize=(12, 4))
ax.set_aspect('equal')
ct = ax.contourf(Y, Z, gfr(Bz_on_mesh, Bz0).reshape(*Y.shape), levels=64)
plt.colorbar(ct)
ax.set_xlabel('x (mm)')
ax.set_ylabel('y (mm)')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
fig, ax = plt.subplots(1,1, figsize=(12, 4))

ax.set_title("Error < 1e-4")
ax.set_aspect('equal')
ct = ax.contourf(Y, Z, gfr(Bz_on_mesh, Bz0).reshape(*Y.shape) < 1e-4, levels=64)
# plt.colorbar(ct)
ax.set_xlabel('x (mm)')
ax.set_ylabel('y (mm)')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Integrated Multipole Content

In [24]:
def harm(obj, y, z, R, harmonics, start=-10, end=10, integral_setting = 'inf'):
    arHarm = []
    angles = np.linspace(0, 2 * np.pi, harmonics, endpoint=False)
    for angle in angles:
        cosTet = np.cos(angle)
        sinTet = np.sin(angle)
        re = rad.FldInt(obj, integral_setting, 'ibz', [start, y + R * cosTet, z + R * sinTet],
                        [end, y + R * cosTet, z + R * sinTet])
        im = rad.FldInt(obj, integral_setting, 'iby', [start, y + R * cosTet, z + R * sinTet],
                        [end, y + R * cosTet, z + R * sinTet])
        arHarm.append(complex(re, im))
    return arHarm


def multipole(arHarm, R, n):
    s = 0
    nharm = len(arHarm)
    for i, harm in enumerate(arHarm):
        arg = -2 * np.pi / nharm * n * i
        s += harm * complex(np.cos(arg), np.sin(arg))
    return s / nharm / (R ** n)

In [26]:
# harmonic analysis of the field integrals
nharm = 8; radius = 0.25; y = 0; z = 0;
w = harm(dipole, y, z, radius, nharm);

mm = [multipole(w, radius, i) for i in range(nharm)];
round_mm = [complex(round(mm[i].real, 9), round(mm[i].imag, 9)) for i in range(nharm)];

In [27]:
print(round_mm)

[(-16.771695444+5.439e-06j), (-1.7939e-05-5.92e-07j), (-0.230239497-2.923e-06j), (8.777e-06+5.28e-07j), (0.107409174+1.932e-06j), (-9.138e-06-5.88e-07j), (-0.108089422-2.277e-06j), (1.1482e-05+9e-07j)]


In [28]:
xxx = round_mm[0]

In [29]:
X = np.arange(nharm)
fig, ax = plt.subplots()
re_comp = [ harms.real for harms in round_mm]
im_comp = [ harms.imag for harms in round_mm]
ax.bar(X + 0.00, np.abs(re_comp), color = 'b', width = 0.25)
ax.bar(X + 0.25, np.abs(im_comp), color = 'g', width = 0.25)
ax.set_yscale('log')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [121]:
# harmonic analysis of the field integrals
n = 8 
radius = 0.25
y = 0 
z = 0
w = harm2(dipole, y, z, radius, 256)

In [122]:
multipole2(w, radius, 0)

(-17.08712515861632+5.446021283390511e-06j)

In [128]:
n = 8 
radius = 0.25
y = 0 
z = 0
for dist in np.logspace(-5, 8, 14):
    print(dist)
    w = harm2(dipole, y, z, radius, 256, start=-dist, end=dist)
    print(multipole2(w, radius, 0))

1e-05
(-16.771697615942458+5.441661314820956e-06j)
0.0001
(-16.77169763031978+5.441812137009028e-06j)
0.001
(-16.771891315761984+5.439526175799857e-06j)
0.01
(-16.76818789353493+5.412174832843293e-06j)
0.1
(-16.77169771051664+5.441864331977644e-06j)
1.0
(-16.771697617817484+5.4418637429191116e-06j)
10.0
(-16.771697616807828+5.441866493957441e-06j)
100.0
(-16.771697620093075+5.441861573915853e-06j)
1000.0
(-16.771697949683983+5.4418649257160275e-06j)
10000.0
(-16.771730907504256+5.441867516704866e-06j)
100000.0
(-16.775025060686772+5.442099299328993e-06j)
1000000.0
(-17.08712512633519+5.438099444675772e-06j)
10000000.0
(-20.598493793569496+5.444275247692759e-06j)
100000000.0
(-20.95836837663297+5.43816751158205e-06j)


In [ ]:
n = 8 
radius = 0.25
y = 0 
z = 0
for dist in np.logspace(-5, 8, 14):
    print(dist)
    w = harm2(dipole, y, z, radius, 256, start=-dist, end=dist, integral_setting = 'fin')
    print(multipole2(w, radius, 0))

1e-05
(-3.875393652719534e-05-1.4031327669394773e-11j)
0.0001
(-0.0003875394212913593-1.4058681096884014e-10j)
0.001
(-0.003875395019364175-1.4058738936253928e-09j)
0.01
(-0.038753947396005226-1.4058551778678758e-08j)
0.1
(-0.387536729842386-1.4048405120102678e-07j)
1.0
(-3.877224412964276-1.2470050063337367e-06j)
10.0
(-17.617963287319967+5.837755926984926e-06j)
100.0
(-16.77039271221385+5.4326485582712e-06j)
1000.0
(-16.761736457615935+5.423712274248973e-06j)
10000.0
